# Creation of a simple wildfire prediction model
#### _A model to get the probability of a wildfire starting in a state of given parameter_

## Finding the right parameter
- Average wind speed
- Wind direction
- Humidity index
- Average temperature
- State
- County

Data source : [Global Surface Summary of the Day - GSOD](https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc%3AC00516/html#)
API : https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation
## Building a source dataset

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import os
import numpy as np

## 1 - Filtering data from other country

In [3]:
errorfile = []
station2keep = []
listoffiles = os.listdir('1992')
for file in listoffiles:
    coordinate = (pd.read_csv(f'1992/{file}')['LATITUDE'][0].astype(str), pd.read_csv(f'1992//{file}')['LONGITUDE'][0].astype(str))
    if coordinate[1] != 'nan':
        geolocator = Nominatim(user_agent="http")
        location = geolocator.reverse(coordinate)
        if location is not None:
            if 'country_code' in location.raw['address'].keys():
                if geolocator.reverse(coordinate) != None:
                    if (str(location.raw['address'].keys()) == 'dict_keys([\'man_made\'])') | (location.raw['address']['country_code'] != 'us'):
                        os.remove(f"1992/{file}")
                    else:
                        print(location.raw['address']['country_code'])
                else:
                    errorfile.append(f'1992/{file}')
            else:
                os.remove(f"1992/{file}")
        else:
            os.remove(f"1992/{file}")
    else:
        errorfile.append(f'1992/{file}')

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=-22.018431&lon=35.313297&format=json&addressdetails=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9dd73fbd90>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [21]:
location is None

True

In [ ]:
listoffiles_source = open(r"1992/listoffiles.txt", "r") #geting the list of files for the year
listoffiles = listoffiles_source.readlines() #geting all the lines
listoffiles = [line[:-1] for line in listoffiles] #deleting the return

In [17]:
column2drop = ['SOURCE', 'ELEVATION', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL', 'MD1', 'MW1', 'OA1', 'OA2', 'SA1', 'UA1', 'UG2', 'WA1', 'REM', 'EQD', 'AA1', 'AA2', 'AG1', 'AJ1', 'AY1', 'AY2', 'GA1', 'GA2', 'GA3', 'GF1', 'IA1', 'KA1', 'KA2', 'MA1']

In [18]:
pd.read_csv('1992/01001099999.csv').drop(column2drop, axis=1)

,STATION,DATE,LATITUDE,LONGITUDE,NAME,WND,CIG,VIS,TMP,DEW,SLP
0,1001099999,1992-01-01T00:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","010,1,N,0093,1","01200,1,C,N","040000,1,N,9","-0036,1","-0077,1","09688,1"
1,1001099999,1992-01-01T03:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","360,1,N,0082,1","00900,1,C,N","020000,1,N,9","-0055,1","-0084,1","09695,1"
2,1001099999,1992-01-01T06:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","360,1,N,0103,1","00900,1,C,N","010000,1,N,9","-0079,1","-0115,1","09709,1"
3,1001099999,1992-01-01T09:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","360,1,N,0113,1","00900,1,C,N","006000,1,N,9","-0086,1","-0109,1","09721,1"
4,1001099999,1992-01-01T12:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","350,1,N,0093,1","00900,1,C,N","004000,1,N,9","-0082,1","-0105,1","09743,1"
...,...,...,...,...,...,...,...,...,...,...,...
2860,1001099999,1992-12-31T09:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","350,1,N,0057,1","02400,1,C,N","040000,1,N,9","-0010,1","-0048,1","09992,1"
2861,1001099999,1992-12-31T12:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","350,1,N,0134,1","02400,1,C,N","040000,1,N,9","-0030,1","-0105,1","09984,1"
2862,1001099999,1992-12-31T15:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","350,1,N,0087,1","06000,1,C,N","040000,1,N,9","-0041,1","-0115,1","09995,1"
2863,1001099999,1992-12-31T18:00:00,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO","360,1,N,0026,1","02400,1,C,N","030000,1,N,9","-0042,1","-0110,1","10002,1"


In [11]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="http")
location = geolocator.reverse("70.933333, -8.666667")
print(location.raw['address']['country'])


Norge



## Publishing the model online